
# Self Flying Drone Using Ryze Tello Drone
A Ryze Tello Drone is required for this part of the project.

## Objectives:

- Return bounding box around face
- Classify face as mine or not
- Calculate velocities required to track face
- Send RC controls to the drone [velocities].
 
    
## Improvements:

Improvements made on the original project by Jabrills: https://www.youtube.com/watch?v=esw88_gKOpA

- Auto-land if no face is detected for certain duration of time
- Search for the face to track by panning and moving up and down
- Implement a fail safe key
- Implement face recognition to ignore unwanted faces. Useful when used around other people

    
##### Comments:

- Initially tried to convert pixel values to centimeters which was unecessary and only accurate in the z-axis between 30cm and 150cm
- Thought the face recognition model would be too slow however it performs fast enough
- Used a counter instead of time.time()-kill to determine how long it had been since the last face was detected
      
## Instructions:

*** If you have not set up the facial recognition please do before performing these steps. This can be done using the file "Create Training Data and Train Model.ipynb". The file contains the relevant setup instructions.***
    
1. Start with the Drone Wi-Fi Disconnected (It causes issues when trying to import modules if it's connected)
2. Import the modules
    2.1 (Importing djitellopy before connecting to the drone wifi causes the program to freeze)
3. Connect to the drone Wi-Fi
4. Place the drone on a flat surface that is the same level as your feet.
5. Run all the code up to and including the flight loop
6. To land, spam the Esc key
    - If the landing zone is not safe for landing. Place your hand under the drone
    - If the drone crash lands; it may be necessary to restart the kernel and reset the drone.
    - If the drone lands succesfully; to take off again, simply run the flight loop.
    - Resetting the drone and restarting the kernel should fix all connectivity issues
    
#### Remarks

not_face_detected() is not finished yet.

### Variables that control drone and program behviour

#### Video Stream

- Scale Factor: Adjust the video stream size without affecting the movement calculations and grid placement
- FPS: How many times the drone sends a captured frame every second 
- Show Stream: True or False
    
#### Movement 

- Green Zone: Region in which recentering is not required   
- Z Zero: The 0 point on the z-axis. When z is equal to this value, the drone holds it's position. 
- Z Forward: Closest that drone can be to face
- Z Backward: Furthest drone can be from the face    
- Up-down(Not detected): Velocity of up and down movemnt to search for face.
    
#### Display

- Display Info: Text and grid information displayed on video feed
- Width: Width of the video stream
- Height: Height of the video stream
- Resolution: This is adjusted using the width and height
- Center: The center of the video stream, used to calculate movement required to recenter face

In [ ]:
import cv2
import numpy as np
import time
from djitellopy import Tello
import tensorflow as tf
import sys
import io

In [ ]:
tello = Tello()

In [ ]:
#Video and Display
scale_factor = 1 # This is the inverse scale so scale_factor = 1/scale
width = int(960/scale_factor)
height = int(720/scale_factor)
resolution = (width,height)
center = (int(resolution[0]/2),int(resolution[1]/2))
FPS = 24
show_stream = False

#Movement
green_zone = 150/scale_factor
z_zero = 125
z_forward = 110/scale_factor
z_backward = 140/scale_factor
not_detected_up_down_velocity = 15

#Face Detection and Recognition
model = tf.keras.models.load_model("Face-Recognition.model")
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

#Dipsplay info on video caputre e.g grid lines
font = cv2.FONT_HERSHEY_SIMPLEX
top_text = (50,50)
y_info = (50,100)
x_info = (50,150)
z_info = (50,200)
bat_info = (50,250)
bottom_text = (50, 715)
text_colour = (0,255,255)
face_box_colour = (255,255,255)
error_box_colour = (0,255,0)
grid_colour = (0,255,255)
grid_colour_2 = (0,255,0)
grid_thickness = 3
default_thickness = 10
y_min = int(resolution[1]/2-green_zone/2)
y_max = int(resolution[1]/2+green_zone/2)
x_min = int(resolution[0]/2-green_zone/2)
x_max = int(resolution[0]/2+green_zone/2)
line_1_start = (0,y_min)
line_1_end = (resolution[0],y_min)
line_2_start = (0,y_max)
line_2_end = (resolution[0],y_max)
line_3_start = (x_min,0)
line_3_end = (x_min,resolution[1])
line_4_start = (x_max,0)
line_4_end = (x_max,resolution[1])

#Define these variables first to prevent cv2 image caputre giving an error
#This is becuase image caputre is empty in the beginning as the drone doesn't return an initial image fast enough
x_mid = width/2
y_mid = height/2
face = 1
w,h = 1,1
count = 0
face_detected = False



## Functions:

In [2]:
def left_right(x):
    
    """ 
    Returns left right or hold
    Determines x-axis direction for the drone
    """
    if x >= x_min and x <= x_max: return 'hold'
    elif x < x_min: return 'left'
    elif x > x_max: return 'right'
    

def up_down(y):
    
    """
    Returns Up down or hold
    Determines y-axis direction for the drone
    """
    if y >= y_min and y <= y_max: return 'hold'
    elif y < y_min: return 'up'
    elif y > y_max: return 'down'
    
def forward_back(z):
    
    """ 
    Returns forward, back or hold
    Determines z-axis movement for the drone
    """
    if z <= z_forward and z >= z_backward: return 'hold'
    elif z < z_forward: return 'forward'
    elif z > z_backward: return 'back'
    
def convert_z(z):
    
    """ 
    Corrects the z-axis by inverting it
    """
    z = -z+125
    return(z)

def grid():
    
    """
    All the relevant information that is displayed on screen
    """
    cv2.line(img,line_1_start,line_1_end,grid_colour,grid_thickness)
    cv2.line(img,line_2_start, line_2_end,grid_colour,grid_thickness)
    cv2.line(img,line_3_start,line_3_end,grid_colour,grid_thickness)
    cv2.line(img,line_4_start,line_4_end,grid_colour,grid_thickness)
    cv2.circle(img, (int(resolution[0]/2),int(resolution[1]/2)), default_thickness,grid_colour)
    up = up_down(y_mid)
    right = left_right(x_mid)
    forward = forward_back(h)
    cv2.putText(img,"Right %s" %(right),x_info,font, 1,text_colour,2,cv2.LINE_4)
    cv2.putText(img,"Up %s"%(up),y_info,font, 1,text_colour,2,cv2.LINE_4)
    cv2.putText(img,"Forward %s"%(forward),z_info,font, 1,text_colour,2,cv2.LINE_4)
    cv2.circle(img, center, default_thickness ,grid_colour)
    
def xyz_movement(x,y,z):
    
    """
        Compare x, y and z values of the face with the center and sends controls.
        Controls forced within the range of -100 and 100 to the Drone. 
    """
    x_1 = x
    y_1 = y
    x = x - width/2
    y = y - height/2
    
    x = int((x*75)/(width/2))
    y = int((-y*75)/(height/2))
    
    if left_right(x_1) == "hold":
        x = 0
    if up_down(y_1) == "hold":
        y = 0
    if forward_back == "hold":
        z = z_zero

    z = convert_z(z)
    z = int((z*75)/200)
    tello.send_rc_control(0,z,y,x)
    
def not_face_detected():
    """
    Incomplete
    Used to locate the face if it is not detected.
    """
    not_detected_up_down_velocity = - not_detected_up_down_velocity
    tello.send_rc_control(0,0,not_detected_up_down_velocity,50)
    
def prepare(image):
    """
    Prepare image (current frame) to fit the model.
    """
    IMG_SIZE = 100
    new_array = cv2.resize(image, (IMG_SIZE,IMG_SIZE))/255.0
    return     np.array(new_array).reshape(-1,IMG_SIZE,IMG_SIZE,1).astype('float32')

In [3]:
#Suppress the program from printing every single command that it sends
text_trap = io.StringIO()
sys.stdout = text_trap
#Connect to the drone and takeoff
tello.connect()
time.sleep(2)
tello.streamoff()
tello.streamon()
tello.takeoff()
time.sleep(5)
tello.send_rc_control(0,0,75,0)
time.sleep(2)
tello.send_rc_control(0,0,0,0)
kill = time.time()



#Start detecting the face and tracking it
while True:
    
    #Get frame from drone and detect face
    frame_read = tello.get_frame_read()
    myFrame = frame_read.frame
    img = cv2.resize(myFrame, (width,height))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    font = cv2.FONT_HERSHEY_SIMPLEX 
    
    #Model Prediction
    for (x,y,w,h) in faces:
        prediction = model.predict(prepare(gray[y:y+h, x:x+w]))
        if prediction[0][0] < 0.1:
            face_detected = True
            cv2.rectangle(img, (x,y), (x+w,y+h),face_box_colour , 2)
            x_mid = (2*x + w)/2
            y_mid = (2*y + h)/2
    
    if show_stream:
        #For Better performance, set show_stream = False when not testing
        grid()
        cv2.imshow("Img", img)
        cv2.waitKey(300)
     
    if face_detected:
        #Move the drone (or hold position)
        xyz_movement(x_mid,y_mid,h)
        time.sleep(1/FPS)
        kill = time.time()
        
    elif not face_detected:
        #Tell the Drone to stop moving
        tello.send_rc_control(0,0,0,0)
        if time.time() - kill > 15:
            tello.land()
            break
            
    c = cv2.waitKey(7) % 0x100
    if c == 27 or c == 10:
        tello.land()
        break
    #Reset Face Detected
    face_detected = False 


tello.end()
cv2.destroyAllWindows()

sys.stdout = sys.__stdout__

NameError: name 'io' is not defined